### Motion Controls
##### Goal  
Modular Code So blocks for specific functions or buttons
First Identify What functions are necessary for experiment context

#### Features from LabView Code 

Accepts these Inputs:  
IP Address: 192.168.254.254  
Port No: 5001  
Timeout: 10,000  
GroupName: Group 1  
nbitems: 1  
TargetPosition:  
STOP BUTTON  

### Connect and Status

1. Create xps object and Checks group status below

In [1]:
%load_ext autoreload
%autoreload 2   

In [2]:
import sys
import os

# sys.path.append(r'C:\Users\ooa56\Documents\PLD_ControlSystem_Python')
sys.path.append(r'C:\Users\ooa56\Documents\PLD_ControlSystem_Python\PLD_ControlSystem_Python\src')

In [ ]:

# from src.newportxps import NewportXPS

# xps = NewportXPS('192.168.254.254', username='Administrator', password='Administrator')

# print(xps.status_report())


In [ ]:
# xps.read_systemini()

Error Description:
ZM high level: This refers to the mechanical zero signal used with some stages. The ZM
signal is high during one part of the travel and low during the other part of the travel.
The detection of the ZM high/low transition in combination with an encoder index pulse
signal allows a fast and repeatable origin search
(MechanicalZeroAndIndexHomeSearch).

Do I need to resolve this error? Don't think so has not prevented the running of following commands



In [ ]:
# Print all stages configured on the controller
#for sname, info in xps.stages.items():
#    print(sname, xps.get_stage_position(sname), info)

Note:  To prevent following Error during Homing process. Ensure that within the stages.ini configuration file the Home Search Process Parameters are as below the maximum velocity is 40 Unit/Sec.  
SETTINGS:  
; --- Home search process parameters  
; --- <HomeSearch.MechanicalZeroAndIndexHomeSearch>  
HomeSearchSequenceType = MechanicalZeroAndIndexHomeSearch  
HomeSearchMaximumVelocity = 30 ; Unit/Sec  
HomeSearchMaximumAcceleration = 400 ; Unit/Sec²  
HomeSearchTimeOut = 10 ; Sec  
HomingSensorOffset = 0 ; Unit  

### Initialize and Home

2. Now Initialize Group  
Notes:  
Initializing the group (turning the motor on) causes the stage to move.  
Have not neccessarily observed pattern in how much the stage moves or even the direction it moves.  
Comparing the command above and below to check the position of the group shows the change.
AFter initializing group Status is in Not Referenced State


In [ ]:
#if group status enabled then kill all groups else initialize group
# Remember to Kill Group before exiting or if the motor is still on
#xps.kill_group('Group1')

# Turns on the servo Motor for the specified group and home the stage
#xps.initialize_group('Group1', with_encoder=True, home=True, with_raise=True)

In [ ]:
# Print all groups configured on the controller
#for gname, info in xps.groups.items():
#    print(gname, info)

In [ ]:
# Note Turning the servo on will cause the stage to move the stage position
# Check to make sure that these functions perform as expected
#print('Where Group1.Pos is', xps.get_stage_position('Group1.Pos'), info)
# print(xps.get_group_status())

### Move Motion Stage

In [ ]:
# Move the stage to a new position. Default is absolute motion can optionally set the motion to relative by relative=True
# xps.move_stage('Group1.Pos', 10.0)


## start here
#### TESTING FUNCTIONS FROM SIMPLIFIED CONTROLLER SCRIPT

In [3]:
import pld_controlsystem_python.motion_ctrl as mc

In [4]:
# Initialize motion controller with host IP address and optional group name
controller = mc.MotionController('192.168.254.254', username='Administrator', password='Administrator')


The initialize and Home function Initializes and homes all the groups connected to the controller.

In [5]:

# Initialize and home the motion controller
controller.initialize_and_home()


'# XPS host:         192.168.254.254 (XPS-1871)\n# Firmware:         XPS-RL-D-N16002\n# Current Time:     Mon Jun 10 09:54:46 2024\n# Last Reboot:      Fri May 24 16:51:26 2024\n# Trajectory Group: None\n# Groups and Stages\nGroup1 (singleaxisinuse), Status: Ready state from homing\nStages:\n   Group1.Pos (ILS@ILS200LM@XPS-DRV02)\n      Hardware Status: First driver powered on - ZM low level\n      Positioner Errors: OK'

The set_position function currently needs the name of the Stage to be specified but I can hard code this to be the first stage name  this to be the first stage name.

In [6]:

# Set position of a stage
controller.set_position(stage="Group1.Pos", position=50.0)


The current position of Group1.Pos is 50.00587


In [7]:
controller.get_position(stage="Group1.Pos")

The current position of Group1.Pos is 50.00007


In [8]:

# Set velocity parameters for a stage
# Can be expanded to include acceleration and jerktime parameters
controller.set_velocity(stage="Group1.Pos", velocity=500.0, )


The current velocity of Group1.Pos is 500 Units/sec


In [9]:
controller.get_velocity(stage="Group1.Pos") 

The current velocity of Group1.Pos is 500 Units/sec


In [10]:
controller.set_position(stage="Group1.Pos", position=0.0)

The current position of Group1.Pos is 0.00925


In [12]:

# Stop the motion controller
controller.stop_controller()

## Attenuator Controls

In [13]:
import pld_controlsystem_python.attenuator_ctrl as ac


## Troubleshooting attempt ##
1. Take Pictures of current state of controller wiring
2. Examine Attenuator Schematics understand them in relation to the physical system

Today: First tried to run commands above again and got the same response of the cell continuously running. Then tried uploading the Arduino sketch to the controller through the Arduino IDE unsuccessfully.
Will proceed with initial plan to resolve the unresponsiveness of the controller.
06/03/24- Problem Resolved. Created new python script with attenuator controls based on Labview set-up. Gained better undertanding of the labview blocks. Attenuator now responds to commands.

In [14]:
laser = ac.AttenuatorControls()


In [15]:
angle = 45
speed = 4


In [16]:
# Execute the sequence of operations
laser.rotate_to_angle(angle)


In [17]:
laser.clear_laser()


In [18]:
laser.block_laser()


In [19]:
laser.home_attenuator()
laser.set_rotation_speed(speed)

# Close the serial connection
laser.close()


## Target Controls

The carousel responds to commands.

In [20]:
import pld_controlsystem_python.target_ctrl as tc

In [21]:
carousel = tc.TargetControls(port='COM7')



In [22]:

angle = 45
speed = 5


In [23]:
# Execute the sequence of operations
carousel.step_raster_cw()

In [24]:
carousel.step_raster_ccw()

In [25]:
carousel.home_raster()


In [26]:
carousel.set_raster_speed(speed)


In [27]:
carousel.start_raster(45)


In [28]:
carousel.stop_raster()


In [29]:
carousel.start_rotate()


In [30]:
carousel.stop_rotation()


In [31]:
carousel.rotate_to_angle(angle)
carousel.set_rotation_speed(speed)
carousel.move_to_target(1)

# Close the serial connection
carousel.close()
 

## Vacuum Controls


In [32]:
import pld_controlsystem_python.vacuum_ctrl as vc


In [33]:
vac = vc.VacuumControls()

In [34]:
vac.initialize()

Initialization successful


In [40]:
response_mbar = vac.read_data(parameter=4)

In [36]:
if response_mbar:
    response_mbar = float(response_mbar)
    response_torr = vc.mbar_to_torr(response_mbar)
    print(f'Response: {response_mbar} mbar')
    print(f'Response: {response_torr} Torr')